---
title: Knuckledragger 3
---

Untyped terms feels kind of like a toy. It is super useful for tinkering and if you know what you're doing it is less constrained. But, it is way easier to make mistakes in your formulation
https://tptp.org/TPTP/SyntaxBNF.html

In [28]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Sort:
    def __mul__(self, rhs):
        return TProd((self,rhs))
    def __rshift__(self,rhs):
        return TFun(self, rhs)

@dataclass(frozen=True)
class TConst(Sort):
    name: str
    args: tuple[Sort] = ()
    #def __repr__(self):
    #    if self.args:
    #        return f'{self.name}({", ".join(map(repr, self.args))})'
    #    return self.name
    def thf(self):
        return self.name

IntSort = TConst('$int')
BoolSort = TConst('$o')
RealSort = TConst('$real')
RatSort = TConst('$rat')
IndSort = TConst("$i")

@dataclass(frozen=True)
class TFun(Sort):
    input: Sort
    output: Sort
    def thf(self):
        return f"({self.input.thf()} > {self.output.thf()})"

@dataclass(frozen=True)    
class TProd(Sort):
    args: tuple[Sort, ...]
    def thf(self):
        return "(" + " * ".join(a.thf() for a in self.args) + ")" # Hmm what about empty or single tuples?


def TFun1(*args):
    args = list(args)
    a = args.pop()
    while len(args) > 0:
        b = args.pop()
        a = TFun(b,a)
    return a

"""
def TFun(*args):
    if len(args) == 1:
        return args[0]
    else:
        return TConst('>', (args[0], TFun(*args[1:])))
"""

def thf_type_decl(t : Sort):
    return f'thf(type_decl,type,{t.thf()}).'

def test():
    a = TConst("a")
    b = TConst("b")
    print((a * b).thf())
    print((a >> b >> a).thf())
    print(TFun1(a,a,a,a,b).thf())

test()

(a * b)
((a > b) > a)
(a > (a > (a > (a > b))))


Terms are like before, but we tag them with sorts.

In [37]:
@dataclass(frozen=True)
class Term:
    def __eq__(self : "Term", other : "Term"):
        return Eq(self, other)
    def __call__(self, *args):
        res = self
        for a in args:
            res = Apply(res, a)
        return res
    def __matmul__(self, other):
        return Apply(self, other)

@dataclass(frozen=True)
class Var(Term):
    name: str
    ty : Sort
    def __repr__(self):
        return self.name.title()
    def vars(self):
        return {self}

@dataclass(frozen=True)
class Fn(Term):
    name: str
    args: tuple[Term, ...]
    ty: Sort
    def __eq__(self : Term, other : Term):
        return Eq(self, other)
    def vars(self):
        return {v for a in self.args for v in a.vars()}
    def thf(self):
        if len(self.args) == 0:
            return self.name
        return f"{self.name}({', '.join(a.thf() for a in self.args)})"

@dataclass(frozen=True)
class InFix(Fn):
    def thf(self):
        return f"({self.args[0].thf()} {self.name} {self.args[1].thf()})"


# How to do ty annotation?
@dataclass(frozen=True)
class Lambda(Term):
    args: tuple[Var, ...]
    body: Term 
    @property
    def ty(self):
        args = tuple(a.ty for a in self.args) + (self.body.ty,)
        return TFun(*args)
    def thf(self):
        vars = ', '.join([ f"{v} : {v.ty}" for v in self.args])
        return f"^[{vars}] : ({self.body})"

@dataclass(frozen=True)
class Let(Term):
    bindings: tuple[tuple[Var, Term], ...]
    body: Term
    @property
    def ty(self):
        return self.body.ty
    def thf(self): # TODO
        return f"$let({self.bindings}, {self.body})"

# helpers
def Const(name : str, ty : Sort):
    return Fn(name, (), ty)
def Consts(names : str, ty : Sort):
    return [Const(name, ty) for name in names.split()]
def Function(name : str, *ty):
    """def res(*args):
        assert len(args) == len(ty) - 1
        assert all(a.ty == b for a,b in zip(args, ty[:-1])) # a bit weird to do this here...
        return Fn(name, args, ty[-1])
    return res"""
    assert all(isinstance(ty, Sort) for ty in ty)
    return Const(name, TFun1(*ty))

# built ins
def Ite(c,t,e):
    return Fn('$ite', (c,t,e), BoolSort)
# Should these be special?
def Apply(f : Term, x : Term):
    assert isinstance(f.ty, TFun)
    assert f.ty.input == x.ty
    return InFix('@', (f,x), f.ty.output)
def Eq(x : Term, y : Term):
    return InFix('=', (x, y), BoolSort)

def test():
    a = TConst("a")
    b = TConst("b")
    x = Const("x", a)
    print(x.thf())
    f = Function("f", a , a, a)
    print(x.ty)
    print(f)
    f @ x
    f(x)
    a = TConst("a")
    x,y,z = Consts("x y z", a)
    f = Function("f", a, a)
    f(f(x))
    b = TConst("b")
    w = Const("w", b)
    #f(w)
    X = Var("X", a)

    print(Lambda([X], f(X))(x))
    e = x == y
    e.thf()

    
test()

x
TConst(name='a', args=())
Fn(name='f', args=(), ty=TFun(input=TConst(name='a', args=()), output=TFun(input=TConst(name='a', args=()), output=TConst(name='a', args=()))))
InFix(name='@', args=(Lambda(args=[X], body=InFix(name='@', args=(Fn(name='f', args=(), ty=TFun(input=TConst(name='a', args=()), output=TConst(name='a', args=()))), X), ty=TConst(name='a', args=()))), Fn(name='x', args=(), ty=TConst(name='a', args=()))), ty=TConst(name='a', args=()))


In [8]:
def type_infer(t):
    pass
def type_check(t):
    pass

Helpers

IndexError: tuple index out of range

We can really bulk out our special formula constructors. I kind of hate this. It's so verbose. But it is probably what the people crave


In [ ]:
@dataclass(frozen=True)
class Formula:
    @property
    def ty(self):
        return BoolSort
    def __eq__(self):
        return Eq(self, other)

@dataclass(frozen=True)
class FTrue(Formula):
    def thf(self):
        return "$true"

@dataclass(frozen=True)
class FFalse(Formula):
    def thf(self):
        return "$false"
    

@dataclass(frozen=True)
class Not(Formula):
    arg: Formula
    def thf(self):
        return f"~({self.arg.thf()})"

@dataclass(frozen=True)
class And(Formula):
    x: Formula
    y: Formula
    def thf(self):
        return f"({self.x.thf()} & {self.y.thf()})"

@dataclass(frozen=True)
class Or(Formula):
    x: Formula
    y: Formula
    def thf(self):
        return f"({self.args[0].thf()} | {self.args[1].thf()})"

@dataclass(frozen=True)
class Implies(Formula):
    hyp: Formula
    conc: Formula
    def thf(self):
        return f"({self.hyp.thf()} => {self.conc.thf()})"

@dataclass(frozen=True)
class Iff(Formula):
    lhs: Formula
    rhs: Formula
    def thf(self):
        return f"({self.lhs.thf()} <=> {self.rhs.thf()})"

@dataclass(frozen=True)
class ForAll(Formula):
    var: str
    body: Formula
    def thf(self):
        return f"! [ {self.var} ] : ({self.body.thf()})"

@dataclass(frozen=True)
class Exists(Formula):
    var: str
    body: Formula
    def thf(self):
        return f"? [ {self.var} ] : ({self.body.thf()})"

@dataclass(frozen=True)
class Atom(Formula):
    pred: str
    args: tuple[Term]


@dataclass(frozen=True)
class Eq(Formula):
    lhs: Term
    rhs: Term
    def thf(self):
        return f"({self.lhs.thf()} = {self.rhs.thf()})"



In [ ]:
f = And(FTrue(),FFalse())
f.thf()
f.ty
FTrue() == FFalse()

False

In [ ]:
def declare_type(ty : Sort):
    return f"thf({ty.name}_decl,type,{ty}: $tType).\n"
def declare_const(c : Term):
    return f"thf({c.name}_decl, type, {c.name} : {c.ty}).\n"
import eprover
class Theory(): ## ProofDB
    def __init__(self):
        self.__proofdb = []
        self.sig = {} # definitions?
        self.types = {}
    def last_theorem(self):
        return Theorem(len(self.__proofdb) - 1, self)
    def assume(self, t : Formula):
        assert isinstance(t, Formula)
        self.__proofdb.append((t, "assume"))
        return self.last_theorem()
    def infer(self, conc: Formula, by=[], timeout=1.0, save_proof=False):
        # could be neat to make hyps optional and just toss proof_db to the solver if hyps=None
        hyps = by
        assert all(isinstance(hyp, Theorem) and hyp.proofdb == self for hyp in hyps)
        assert isinstance(conc, Formula)
        with open("/tmp/myfile.p", "w") as f:
            for typ in self.types.values():
                f.write(declare_type(typ))
            for const in self.sig.values():
                f.write(declare_const(const))
            for hyp in hyps:
                print(hyp.thf())
                f.write(f"thf(thm{hyp.index},axiom, {hyp.thf()}).\n")
            f.write(f"thf(goal, conjecture, {conc.thf()}).\n")
        res = eprover.run( ["/tmp/myfile.p"], timeout=timeout, capture_output=True)
        if "SZS status Theorem" not in res.stdout.decode("utf-8"):
            raise Exception(f"Proof failed \n{hyps}\n----------------(eprover)\n{conc}\n", res.stdout.decode("utf-8"), res.stderr.decode("utf-8")) 
        if save_proof:
            self.__proofdb.append((conc, ("infer", hyps, res.stdout)))
        else:   
            self.__proofdb.append((conc, ("infer", hyps)))
        return self.last_theorem()
    def __getitem__(self, key):
        return self.__proofdb[key]
    def make_type(self,name):
        t = TConst(name)
        self.types[name] = t
        return t
    def make_const(self,name,ty):
        c = Const(name,ty)
        self.sig[name] = c
        return c
# It isn't persay such an issue that theorem's constructor is not protected because the proof db holds the only reference.
@dataclass(frozen=True)
class Theorem():
    index: int
    proofdb: Theory
    def formula(self):
        return self.proofdb[self.index][0]
    def cnf(self):
        return self.formula().cnf()
    def thf(self):
        return self.formula().thf()
    def __repr__(self):
        return f"Theorem({self.formula()})"
    


t = Theory()
a = t.make_type("a")
x = t.make_const("x", a)
y = t.make_const("y", a)
z = t.make_const("z", a)
#x == y
p = t.assume(x == y)
q = t.assume(y == z)
t.infer(x == z, by=[p,q])

(x = y)
(y = z)


Theorem(Eq(lhs=Fn(name='x', args=(), ty=a), rhs=Fn(name='z', args=(), ty=a)))

https://tptp.org/Seminars/TPTPWorldTutorial/LogicTHF.html

In [ ]:
types = [
    TConst("a")
]







In [ ]:
%%file /tmp/test.p
%------------------------------------------------------------------------------
thf(beverage_decl,type,beverage: $tType).
thf(syrup_decl,type,syrup: $tType).
thf(coffee_decl,type,coffee: beverage).
thf(mix_decl,type,mix: beverage > syrup > beverage).
thf(heat_decl,type,heat: beverage > beverage ).
thf(heated_mix_decl,type,heated_mix: beverage > syrup > beverage).
thf(hot_decl,type,hot: beverage > $o).

thf(heated_mix,axiom,
    heated_mix = ( ^ [B: beverage,S :syrup] : ( heat @ ( mix @ B @ S ))) ).

thf(hot_mixture,axiom,
    ! [B: beverage,S: syrup] : ( hot @ ( heated_mix @ B @ S ) ) ).

thf(heated_coffee_mix,axiom,
    ! [S: syrup] : ( ( heated_mix @ coffee @ S ) = coffee ) ).

thf(hot_coffee,conjecture,
    ? [Mixture: syrup > beverage] :
    ! [S: syrup] :
        ( ( ( Mixture @ S ) = coffee )
        & ( hot @ ( Mixture @ S ) ) ) ).

Writing /tmp/test.p


In [ ]:
!vampire /tmp/test.p

% Running in auto input_syntax mode. Trying TPTP
User error: This version of Vampire is not yet HOLy.

Support for higher-order logic is currently on the ahmed-new-hol branch.
HOL should be coming to mainline 'soon'.


In [ ]:
!zipperposition /tmp/test.p

# done 4 iterations in 0.005s
# SZS status Theorem for '/tmp/test.p'
# SZS output start Refutation
* assert [file "/tmp/test.p" "heated_coffee_mix"]
    ∀ S/54:syrup. ((heated_mix coffee S/54) = coffee). by
  'heated_coffee_mix' in '/tmp/test.p'
* forall (X0:syrup). heated_mix coffee X0 = coffee(2) by
  esa 'cnf'
    with assert [file "/tmp/test.p" "heated_coffee_mix"]
           ∀ S/54:syrup. ((heated_mix coffee S/54) = coffee).
* goal [file "/tmp/test.p" "hot_coffee"]
    ∃ Mixture/55:(syrup → beverage).
      (∀ S/56:syrup.
         ((hot (Mixture/55 S/56)) ∧ ((Mixture/55 S/56) = coffee))). by
  goal 'hot_coffee' in '/tmp/test.p'
* negated_goal
    ¬ (∃ Mixture/55:(syrup → beverage).
          (∀ S/56:syrup.
             ((hot (Mixture/55 S/56)) ∧ ((Mixture/55 S/56) = coffee))))
    # skolems: []. by
  esa 'cnf.neg'
    with goal [file "/tmp/test.p" "hot_coffee"]
           ∃ Mixture/55:(syrup → beverage).
             (∀ S/56:syrup.
                ((hot (Mixture/55 S/56)) ∧ ((Mixt

Wow. tyhat neither of these work is not encouraging.

Changing the name of ProofDb to Theory is interesting.
Theory combinators (maude)
PVS




https://plato.stanford.edu/entries/type-theory-church/
LEO II III https://page.mi.fu-berlin.de/cbenzmueller/leo/ https://github.com/leoprover/Leo-III
Satallax https://satallaxprover.org/
Lash https://arxiv.org/abs/2205.06640 satallax sequel? http://grid01.ciirc.cvut.cz/~chad/lash/

TPS adnrews. etps